In [26]:
%reload_ext autoreload
%autoreload 2
import os
import glob
import scipy.io.wavfile as wv
import matplotlib.pyplot as plt
import numpy as np
from util import util
import scipy.signal as sig
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir

baseTargetDir = '/Users/etto/Desktop/pData'
baseTargetDir = 'E:/SAA_DATA/targetDir'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

NFFT = 1024

In [4]:
# Train alleen op 1 van de org dirs
srcDir = baseSrcDir + '/' + orgWavDirs1[0]
filesPerCategory = {}
for catDirLong in glob.glob(srcDir + '/*'):
    catDir = catDirLong.replace('\\', '/')
    catDir = catDir.replace(srcDir + '/', '')
    
    filesPerCategory[catDir] = []
    for filename in glob.glob(catDirLong + '/*'):
        filename = filename.replace('\\','/')
        filesPerCategory[catDir].append(filename)


In [5]:
# maak spectros van elke file
spectros = {}
for clz in classes:
    spectros[clz] = []
    for filename in filesPerCategory[clz]:
        fs, signal = wv.read(filename)
        freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
        spectros[clz].append(spectrogram.T)
        

In [6]:
#alle X achter elkaar in goede volgorde

X_train = np.concatenate(spectros[classes[0]], axis=0)
for i in range(1, len(classes)):
    nwSpectros = np.concatenate(spectros[classes[i]], axis=0)
    X_train = np.concatenate((X_train,nwSpectros), axis=0) 

np.shape(X_train)

(187316, 513)

In [7]:
clzLengths = {}
for clz in classes:
    clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectros[clz]])
print(clzLengths)

{'music': 60205, 'voice': 60859, 'environment': 66252}


In [8]:
# one-hot encoding
clsNr = 0
nrFiles = clzLengths[classes[0]]
Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

nrFiles = clzLengths[classes[1]]
Y_train_nw = np.array((np.zeros(nrFiles),np.ones(nrFiles), np.zeros(nrFiles))).T
Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

nrFiles = clzLengths[classes[2]]
Y_train_nw = np.array((np.zeros(nrFiles),np.zeros(nrFiles), np.ones(nrFiles))).T
Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
print(np.shape(Y_train))

(187316, 3)


zie https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

In [21]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=513, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [31]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=5, batch_size=5,verbose=1)

In [32]:
seed = 42
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [33]:
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print('Baseline: {:.2f} ({:.2f})'.format(results.mean(), results.std()))

Epoch 1/20
168584/168584 [==============================] - 159s 941us/step - loss: 0.1744 - acc: 0.9369
Epoch 2/20
168584/168584 [==============================] - 158s 936us/step - loss: 0.1115 - acc: 0.9594
Epoch 3/20
168584/168584 [==============================] - 159s 941us/step - loss: 0.0968 - acc: 0.9654
Epoch 4/20
168584/168584 [==============================] - 199s 1ms/step - loss: 0.0887 - acc: 0.9683
Epoch 5/20
168584/168584 [==============================] - 156s 926us/step - loss: 0.0843 - acc: 0.9703
Epoch 6/20
168584/168584 [==============================] - 150s 891us/step - loss: 0.0810 - acc: 0.9711
Epoch 7/20
168584/168584 [==============================] - 156s 927us/step - loss: 0.0758 - acc: 0.9727
Epoch 8/20
168584/168584 [==============================] - 158s 938us/step - loss: 0.0741 - acc: 0.9738
Epoch 9/20
168584/168584 [==============================] - 152s 901us/step - loss: 0.0717 - acc: 0.9745
Epoch 10/20
168584/168584 [==============================

KeyboardInterrupt: 